# connect drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# modify xolotl conf file

In [5]:
%%writefile /kaggle/working/Spark-TTS-finetune_clar/config_axolotl/full_finetune.yml
base_model: pretrained_models/Spark-TTS-0.5B/LLM
# Automatically upload checkpoint and final model to HF

#hub_model_id: Alawy21/spark-tts-finetuned  # Replace with your HF username
#hub_strategy: "checkpoint"                        # Upload strategy: "checkpoint", "end", "every_save"
#hub_token: null                                   # HF token (set via HF_TOKEN env var or login)
#hub_private_repo: true                           # Make repo public (true for private)
#push_dataset_to_hub: "false"                     # Fixed: Changed from false to "false"
#hf_use_auth_token: true                          # Use HF authentication token


trust_remote_code: true
strict: false
datasets:
  - path: /kaggle/input/dataset-v2/ClarTTS_csv.jsonl
    type: completion
    field: text
    
dataset_prepared_path: 
val_set_size: 0.05
output_dir: /kaggle/working/spark_out
sequence_len: 2048
sample_packing: true
eval_sample_packing: true
pad_to_sequence_len: true

use_tensorboard: true
# wandb_project: spark-tts-finetuning
# wandb_watch: all
# wandb_name: spark-tts-t4-run1
# wandb_log_model: "checkpoint"      # Fixed: Changed from true to "true"

# T4-Optimized Settings
gradient_accumulation_steps: 4  # Increased to maintain effective batch size
micro_batch_size: 1             # Reduced for T4's 16GB VRAM
num_epochs: 16
optimizer: adamw_torch_fused
lr_scheduler: cosine
learning_rate: 0.0002
train_on_inputs: false
group_by_length: false

# GPU Architecture Compatibility (T4 = Turing, not Ampere)
bf16: false              # T4 has limited bfloat16 support
fp16: false               # T4 supports FP16 mixed precision
tf32: false              # T4 doesn't support TF32 (Ampere+ only)

# Memory and Performance Optimization
gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: false

# Attention Mechanism (T4 Compatible)
xformers_attention: true    # Works well on T4
flash_attention: false      # T4 doesn't support FlashAttention

# Training Configuration
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 25
warmup_steps: 10
evals_per_epoch: 1
save_steps: 1000
debug:
deepspeed:
weight_decay: 0.0

Overwriting /kaggle/working/Spark-TTS-finetune_clar/config_axolotl/full_finetune.yml


# get the finetune repo 

In [1]:
!git clone https://github.com/ali-adel/Spark-TTS-finetune_clar.git

Cloning into 'Spark-TTS-finetune_clar'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 97 (delta 30), reused 40 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 54.74 KiB | 2.03 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [2]:
%cd /kaggle/working/Spark-TTS-finetune_clar

/kaggle/working/Spark-TTS-finetune_clar


# Install requirements

In [ ]:
!pip install -U packaging setuptools wheel ninja

In [ ]:
!pip install --no-build-isolation axolotl[flash-attn,deepspeed]

In [ ]:
!pip install torch==2.1.2 torchvision==0.16.2 transformers==4.38.2 torchaudio==2.1.2 sentence-transformers==2.2.2 --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install axolotl

In [8]:
%cd /kaggle/working/Spark-TTS-finetune_clar

/kaggle/working/Spark-TTS-finetune_clar


In [ ]:
!python -m src.download_pretrain

In [10]:
%cd /kaggle/working/Spark-TTS-finetune_clar

/kaggle/working/Spark-TTS-finetune_clar


In [ ]:
!pip install flash-attn --no-build-isolation

# get checkpoint from drive to resume from it if needed

In [13]:
# %cd /kaggle/working/
# !gdown 1I7c2_7Z9CUAccHxErGI3K4egXAoZJJK7

/kaggle/working
Downloading...
From (original): https://drive.google.com/uc?id=1I7c2_7Z9CUAccHxErGI3K4egXAoZJJK7
From (redirected): https://drive.google.com/uc?id=1I7c2_7Z9CUAccHxErGI3K4egXAoZJJK7&confirm=t&uuid=993db2d6-e9ee-4c2e-906f-a7edfac64376
To: /kaggle/working/checkpoint-2000.zip
100%|███████████████████████████████████████| 5.60G/5.60G [00:46<00:00, 121MB/s]


In [14]:
#!unzip /kaggle/working/checkpoint-2000.zip -d /kaggle/working/checkpoint-2000

Archive:  /kaggle/working/checkpoint-2000.zip
  inflating: /kaggle/working/checkpoint-2000/tokenizer_config.json  
  inflating: /kaggle/working/checkpoint-2000/training_args.bin  
  inflating: /kaggle/working/checkpoint-2000/merges.txt  
  inflating: /kaggle/working/checkpoint-2000/vocab.json  
  inflating: /kaggle/working/checkpoint-2000/special_tokens_map.json  
  inflating: /kaggle/working/checkpoint-2000/optimizer.pt  
  inflating: /kaggle/working/checkpoint-2000/model.safetensors  
  inflating: /kaggle/working/checkpoint-2000/rng_state_0.pth  
  inflating: /kaggle/working/checkpoint-2000/config.json  
  inflating: /kaggle/working/checkpoint-2000/scheduler.pt  
  inflating: /kaggle/working/checkpoint-2000/added_tokens.json  
  inflating: /kaggle/working/checkpoint-2000/rng_state_1.pth  
  inflating: /kaggle/working/checkpoint-2000/trainer_state.json  
  inflating: /kaggle/working/checkpoint-2000/generation_config.json  
  inflating: /kaggle/working/checkpoint-2000/tokenizer.json  


In [12]:
%cd /kaggle/working/Spark-TTS-finetune_clar

/kaggle/working/Spark-TTS-finetune_clar


In [ ]:
!axolotl train config_axolotl/full_finetune.yml

In [64]:
#!rm /kaggle/working/checkpoint-2500.zip

In [56]:
#%cd /kaggle/working/

/kaggle/working


# to zip folder to upload on drive

In [20]:
import shutil

# Path to the folder you want to zip
folder_to_zip = '/kaggle/working/spark_out/runs'  # Change this to your folder path
# Output zip file (without .zip extension)
output_zip_path = '/kaggle/working/runs_1000_best'  # This will create my_folder_backup.zip

# Create the zip archive
shutil.make_archive(output_zip_path, 'zip', folder_to_zip)

print(f"Folder zipped successfully to {output_zip_path}.zip")


Folder zipped successfully to /kaggle/working/runs_1000_best.zip


# upload on drive using google cloud services

In [21]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from googleapiclient.http import MediaFileUpload
import os

# Configuration
SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = '/kaggle/input/best-api/best_api.json'
PARENT_FOLDER_ID = "17IB8tq62wZJDUlOIL74BAgsXU35I-VrL"

def authenticate():
    """Authenticate with Google Drive API"""
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return creds

def upload_zip_file(zip_file_path):
    """Upload ZIP file to Google Drive"""
    try:
        # Check if file exists
        if not os.path.exists(zip_file_path):
            print(f"❌ File not found: {zip_file_path}")
            return None
        
        # Authenticate
        creds = authenticate()
        service = build('drive', 'v3', credentials=creds)
        
        # Get file name from path
        file_name = os.path.basename(zip_file_path)
        
        # File metadata
        file_metadata = {
            'name': file_name,
            'parents': [PARENT_FOLDER_ID]
        }
        
        # Upload the file
        media = MediaFileUpload(zip_file_path, 
                              mimetype='application/zip',
                              resumable=True)
        
        print(f"Uploading {file_name} to Google Drive...")
        
        file = service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id,name,size'
        ).execute()
        
        print(f"✅ Successfully uploaded!")
        print(f"   File Name: {file.get('name')}")
        print(f"   File ID: {file.get('id')}")
        print(f"   File Size: {int(file.get('size', 0)) / (1024*1024):.2f} MB")
        
        return file.get('id')
        
    except Exception as e:
        print(f"❌ Error uploading file: {str(e)}")
        return None

# Usage - Replace with your ZIP file path
zip_path = "/kaggle/working/runs_1000_best.zip"  # Change this to your ZIP file path
upload_zip_file(zip_path)

Uploading runs_1000_best.zip to Google Drive...
✅ Successfully uploaded!
   File Name: runs_1000_best.zip
   File ID: 1L6AC9oARlQfNYOCJoLanrJHIMTwM4FHo
   File Size: 0.01 MB


'1L6AC9oARlQfNYOCJoLanrJHIMTwM4FHo'

# List and delete files from google cloud 

In [22]:
def list_files():
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    results = service.files().list(pageSize=100, fields="files(id, name)").execute()
    files = results.get('files', [])
    if not files:
        print("No files found.")
    else:
        print("Files:")
        for file in files:
            print(f"{file['name']} (ID: {file['id']})")

def delete_file(file_id):
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    try:
        service.files().delete(fileId=file_id).execute()
        print(f"Deleted file with ID: {file_id}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [23]:
list_files()

Files:
runs_1000_best.zip (ID: 1L6AC9oARlQfNYOCJoLanrJHIMTwM4FHo)
ck-1000_best.zip (ID: 1iyI1GBdPhKpOFVRwpdq8fT6Hi7VjbGkW)
spark_best (ID: 17IB8tq62wZJDUlOIL74BAgsXU35I-VrL)


In [58]:
delete_file("1nAZrmfhQ-dAajcuu9-JYbpMskYfNNBk-")

Deleted file with ID: 1nAZrmfhQ-dAajcuu9-JYbpMskYfNNBk-
